In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('wine_quality').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/winequality_white.csv',inferSchema=True,header=True,sep=';')

In [0]:
df.printSchema()

root
-- fixed acidity: double (nullable = true)
-- volatile acidity: double (nullable = true)
-- citric acid: double (nullable = true)
-- residual sugar: double (nullable = true)
-- chlorides: double (nullable = true)
-- free sulfur dioxide: double (nullable = true)
-- total sulfur dioxide: double (nullable = true)
-- density: double (nullable = true)
-- pH: double (nullable = true)
-- sulphates: double (nullable = true)
-- alcohol: double (nullable = true)
-- quality: integer (nullable = true)

In [0]:
for wine in df.head(5):
  print(wine)
  print('\n')

Row(fixed acidity=7.0, volatile acidity=0.27, citric acid=0.36, residual sugar=20.7, chlorides=0.045, free sulfur dioxide=45.0, total sulfur dioxide=170.0, density=1.001, pH=3.0, sulphates=0.45, alcohol=8.8, quality=6)


Row(fixed acidity=6.3, volatile acidity=0.3, citric acid=0.34, residual sugar=1.6, chlorides=0.049, free sulfur dioxide=14.0, total sulfur dioxide=132.0, density=0.994, pH=3.3, sulphates=0.49, alcohol=9.5, quality=6)


Row(fixed acidity=8.1, volatile acidity=0.28, citric acid=0.4, residual sugar=6.9, chlorides=0.05, free sulfur dioxide=30.0, total sulfur dioxide=97.0, density=0.9951, pH=3.26, sulphates=0.44, alcohol=10.1, quality=6)


Row(fixed acidity=7.2, volatile acidity=0.23, citric acid=0.32, residual sugar=8.5, chlorides=0.058, free sulfur dioxide=47.0, total sulfur dioxide=186.0, density=0.9956, pH=3.19, sulphates=0.4, alcohol=9.9, quality=6)


Row(fixed acidity=7.2, volatile acidity=0.23, citric acid=0.32, residual sugar=8.5, chlorides=0.058, free sulfur dioxide=47.0, total sulfur dioxide=186.0, density=0.9956, pH=3.19, sulphates=0.4, alcohol=9.9, quality=6)

In [0]:
df.groupBy('quality').count().orderBy('quality').show()

+-------+-----+
quality|count|
+-------+-----+
 3| 20|
 4| 163|
 5| 1457|
 6| 2198|
 7| 880|
 8| 175|
 9| 5|
+-------+-----+

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

Out[8]: ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [0]:
assembler = VectorAssembler(inputCols = ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol'], outputCol = 'features')

In [0]:
output = assembler.transform(df)

In [0]:
output.columns

Out[11]: ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality',
 'features']

In [0]:
output.select('features','quality').show()

+--------------------+-------+
 features|quality|
+--------------------+-------+
[7.0,0.27,0.36,20...| 6|
[6.3,0.3,0.34,1.6...| 6|
[8.1,0.28,0.4,6.9...| 6|
[7.2,0.23,0.32,8....| 6|
[7.2,0.23,0.32,8....| 6|
[8.1,0.28,0.4,6.9...| 6|
[6.2,0.32,0.16,7....| 6|
[7.0,0.27,0.36,20...| 6|
[6.3,0.3,0.34,1.6...| 6|
[8.1,0.22,0.43,1....| 6|
[8.1,0.27,0.41,1....| 5|
[8.6,0.23,0.4,4.2...| 5|
[7.9,0.18,0.37,1....| 5|
[6.6,0.16,0.4,1.5...| 7|
[8.3,0.42,0.62,19...| 5|
[6.6,0.17,0.38,1....| 7|
[6.3,0.48,0.04,1....| 6|
[6.2,0.66,0.48,1....| 8|
[7.4,0.34,0.42,1....| 6|
[6.5,0.31,0.14,7....| 5|
+--------------------+-------+
only showing top 20 rows

In [0]:
final_data = output.select(['features','quality'])

In [0]:
final_data.columns

Out[14]: ['features', 'quality']

In [0]:
final_data.head(3)

Out[15]: [Row(features=DenseVector([7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]), quality=6),
 Row(features=DenseVector([6.3, 0.3, 0.34, 1.6, 0.049, 14.0, 132.0, 0.994, 3.3, 0.49, 9.5]), quality=6),
 Row(features=DenseVector([8.1, 0.28, 0.4, 6.9, 0.05, 30.0, 97.0, 0.9951, 3.26, 0.44, 10.1]), quality=6)]

In [0]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-----------------+
summary| quality|
+-------+-----------------+
 count| 3479|
 mean|5.877551020408164|
 stddev|0.879096708019738|
 min| 3|
 max| 9|
+-------+-----------------+

In [0]:
test_data.describe().show()

+-------+-----------------+
summary| quality|
+-------+-----------------+
 count| 1419|
 mean|5.878787878787879|
 stddev|0.901789286694111|
 min| 3|
 max| 9|
+-------+-----------------+

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
wine_lr = LinearRegression(labelCol='quality')

In [0]:
trained_white_wine = wine_lr.fit(train_data)

In [0]:
results_white_wine = trained_white_wine.evaluate(test_data)

In [0]:
results_white_wine.rootMeanSquaredError

Out[23]: 0.7593721548727337

In [0]:
train_data.describe().show()

+-------+-----------------+
summary| quality|
+-------+-----------------+
 count| 3479|
 mean|5.877551020408164|
 stddev|0.879096708019738|
 min| 3|
 max| 9|
+-------+-----------------+

In [0]:
results_white_wine.r2

Out[25]: 0.29041347818955576

In [0]:
results_white_wine.meanAbsoluteError

Out[26]: 0.5897125363030241

In [0]:
results_white_wine.meanSquaredError

Out[27]: 0.576646069596059

In [0]:
predictions = trained_white_wine.transform(test_data)

In [0]:
predictions.show(truncate=False)

+------------------------------------------------------------+-------+-----------------+
features |quality|prediction |
+------------------------------------------------------------+-------+-----------------+
[4.2,0.215,0.23,5.1,0.041,64.0,157.0,0.99688,3.42,0.44,8.0] |3 |5.062249833190464|
[4.4,0.32,0.39,4.3,0.03,31.0,127.0,0.98904,3.46,0.36,12.8] |8 |6.749324185171048|
[4.7,0.455,0.18,1.9,0.036,33.0,106.0,0.98746,3.21,0.83,14.0]|7 |6.856481546203099|
[4.7,0.785,0.0,3.4,0.036,23.0,134.0,0.98981,3.53,0.92,13.8] |6 |6.193727520388933|
[4.8,0.26,0.23,10.6,0.034,23.0,111.0,0.99274,3.46,0.28,11.5]|7 |6.536233652853213|
[4.8,0.34,0.0,6.5,0.028,33.0,163.0,0.9939,3.36,0.61,9.9] |6 |5.689949111616386|
[5.0,0.24,0.21,2.2,0.039,31.0,100.0,0.99098,3.69,0.62,11.7] |6 |6.586325058964746|
[5.0,0.27,0.32,4.5,0.032,58.0,178.0,0.98956,3.45,0.31,12.6] |7 |6.822959949479667|
[5.0,0.33,0.16,1.5,0.049,10.0,97.0,0.9917,3.48,0.44,10.7] |6 |5.733433897703691|
[5.0,0.455,0.18,1.9,0.036,33.0,106.0,0.98746,3.21,0.83,14.0]|7 |6.876386520108639|
[5.0,0.55,0.14,8.3,0.032,35.0,164.0,0.9918,3.53,0.51,12.5] |8 |6.333647986086845|
[5.1,0.14,0.25,0.7,0.039,15.0,89.0,0.9919,3.22,0.43,9.2] |6 |5.558408490266515|
[5.1,0.21,0.28,1.4,0.047,48.0,148.0,0.99168,3.5,0.49,10.4] |5 |6.07190229044258 |
[5.1,0.23,0.18,1.0,0.053,13.0,99.0,0.98956,3.22,0.39,11.5] |5 |6.145994755341491|
[5.1,0.3,0.3,2.3,0.048,40.0,150.0,0.98944,3.29,0.46,12.2] |6 |6.44424277564778 |
[5.1,0.33,0.22,1.6,0.027,18.0,89.0,0.9893,3.51,0.38,12.5] |7 |6.470510819173512|
[5.1,0.42,0.01,1.5,0.017,25.0,102.0,0.9894,3.38,0.36,12.3] |7 |6.143926548554305|
[5.2,0.155,0.33,1.6,0.028,13.0,59.0,0.98975,3.3,0.84,11.9] |8 |6.73159302693179 |
[5.2,0.2,0.27,3.2,0.047,16.0,93.0,0.99235,3.44,0.53,10.1] |7 |5.973339594676617|
[5.2,0.24,0.15,7.1,0.043,32.0,134.0,0.99378,3.24,0.48,9.9] |6 |5.829593352207951|
+------------------------------------------------------------+-------+-----------------+
only showing top 20 rows

BAD SELECTION - A linear regression is not the right algorith to predict the quaility of wine. A Classification model should be use for this kind of prediction, such as K-Means, Decision Trees or Random Forests